### Construction example
Peter has a construction company. He needs to assign 5 of the company’s teams to work in some of the constructions below:
- A) Revenue of 500, requires 1 team
- B) Revenue of 4,000, requires 3 teams
- C) Revenue of 3,000, requires 2 teams
- D) Revenue of 2,000, requires 1 team
- E) Revenue of 2,000, requires 5 teams


Select the constructions that would maximize the revenue:
- Each construction can be made just once 
- Not all constructions will be selected

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()

m.I = pe.RangeSet(5)
m.x = pe.Var(m.I, domain=pe.Binary)

revs = {
    1: 500,
    2: 4000,
    3: 3000,
    4: 2000,
    5: 2000
}
m.revs = pe.Param(m.I, initialize=revs)

workers = {
    1: 1,
    2: 3,
    3: 2,
    4: 1,
    5: 5
}
m.workers = pe.Param(m.I, initialize=workers)

m.max_workers = pe.Param(initialize=5)


m.cons1 = pe.Constraint(
    expr=(sum(m.x[i]*m.workers[i] for i in m.I) <= m.max_workers)
)

m.obj = pe.Objective(
    expr=(sum(m.x[i]*m.revs[i] for i in m.I)),
    sense=pe.maximize
)

In [3]:
solver = po.SolverFactory('couenne') 
results = solver.solve(m)

In [4]:
m.x.display()

x : Size=5, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary


**Two extra constraints**
- Construction C can only be selected if A is selected
- Construction D can only be selected if A and C are selected

In [5]:
m = pe.ConcreteModel()

m.I = pe.RangeSet(5)
m.x = pe.Var(m.I, domain=pe.Binary)

revs = {
    1: 500,
    2: 4000,
    3: 3000,
    4: 2000,
    5: 2000
}
m.revs = pe.Param(m.I, initialize=revs)

workers = {
    1: 1,
    2: 3,
    3: 2,
    4: 1,
    5: 5
}
m.workers = pe.Param(m.I, initialize=workers)

m.max_workers = pe.Param(initialize=5)


m.cons1 = pe.Constraint(
    expr=(sum(m.x[i]*m.workers[i] for i in m.I) <= m.max_workers)
)
m.cons2 = pe.Constraint(
    expr=(m.x[3] <= m.x[1])
    )

m.cons3 = pe.Constraint(
    expr=(m.x[4] <= m.x[1]*m.x[3])
    ) 

m.obj = pe.Objective(
    expr=(sum(m.x[i]*m.revs[i] for i in m.I)),
    sense=pe.maximize)

In [6]:
solver = po.SolverFactory('couenne') 
results = solver.solve(m)
m.x.display()

x : Size=5, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
